# Normalization

In [ ]:
#load packages
import pandas as pd
import numpy as np
#from my_functions import *
#import pyRserve
import os.path
import gc
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import random
!pip install rpy2==3.5.1
%load_ext rpy2.ipython
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 201 kB 4.7 MB/s 
  Created wheel for rpy2: filename=rpy2-3.5.1-cp37-cp37m-linux_x86_64.whl size=310591 sha256=b2eda22383bf562b5fd0fc69c283053420d1718b39fcb783dd5dc61f5e2bee51
  Stored in directory: /root/.cache/pip/wheels/65/df/9e/39aa22a3f47ec58751b5bf6907036fbc131d85b6d7cbd3b3fe
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5
Mounted at /content/drive


In [ ]:
w%%R
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(c("impute"))
BiocManager::install(c("WGCNA"))
packages = c("RPMM", "sqldf", "WGCNA")
install.packages(packages)
BiocManager::install("GO.db")
BiocManager::install("preprocessCore")
BiocManager::install("GEOquery")
library(GEOquery)
install.packages("stringr")
install.packages("tidyverse")
lapply(packages, require, character.only = TRUE)
packages = c("caret", "glmnet", "MASS")
install.packages(packages)
lapply(packages, require, character.only = TRUE)
library(glmnet)
install.packages("zoo")                                    # Install & load zoo package
library("zoo")


In [ ]:
%%R
normalize=function(data, n="/content/drive/MyDrive/Altum_Age_Dataset/normalization.txt", s="/content/drive/MyDrive/Altum_Age_Dataset/stepwise.txt", mns="/content/drive/MyDrive/Altum_Age_Dataset/21.csv", output="/content/drive/MyDrive/Altum_Age_Dataset/normalized_data.csv",altum=FALSE) 
{

  if(altum){
    temp <- data
    d1 = dim(temp)[[1]]
    d2 = dim(temp)[[2]]
    samples = temp[1:d1, 1:1]
    values = temp[1:d1, 5:d2]
    cpgs = colnames(values)
    mtrx = data.matrix(values)
    mtrx = t(mtrx)
    new_df = data.frame(mtrx)
    new_df = cbind(ID_REF=cpgs, new_df)
    colnames(new_df) <- unlist(append(c("ID_REF"), samples))
    rownames(new_df) <- NULL
    data <- new_df
  }
  

  
  source(n)
  trafo= function(x,adult.age=20) { x=(x+1)/(1+adult.age); y=ifelse(x<=1, log( x),x-1);y }
  anti.trafo= function(x,adult.age=20) { ifelse(x<0, (1+adult.age)*exp(x)-1, (1+adult.age)*x+adult.age) }

  colnames(data)[1] <- "ProbeID"
  d1 = dim(data)[[1]]
  d2 = dim(data)[[2]]
  m=data.frame(Means=rowMeans(data[1:d1, 2:d2]))
  locations = unlist(data$ProbeID)
  probe=read.csv(mns)
  probe = probe[probe$Name %in% locations, ]
  probeAnnotation21kdatMethUsed<<-probe
  
  dat0=data
  dat0=na.aggregate(dat0[1:d1,2:d2])
  dat0=cbind(ProbeID=locations, dat0)
  colnames(dat0)[1] <- "ProbeID"
  nSamples<<-dim(dat0)[[2]]-1
  nProbes<<- dim(dat0)[[1]]
  
  
  dat1 <<- dat0
  
  set.seed(1)
  normalizeData<<-TRUE
  source(s)

  if(altum){
    input = datMethUsedNormalized
    d1 = dim(input)[[1]]
    d2 = dim(input)[[2]]
    ds_age_gender = temp[1:d1, 1:4]
    input = cbind(X=temp[1:d1, 1], dataset=temp[1:d1, 2], age=temp[1:d1, 3], gender=temp[1:d1, 4],  input)
    datMethUsedNormalized = input
  }
  
  write.table(datMethUsedNormalized,output, row.names=F, sep="," )
}

In [ ]:
geo_data_27k = [
    'GSE27317', 
    'GSE41037', 
    'GSE38873', 
    'GSE15745', 
    'GSE32393', 
    'GSE25892', 
    'GSE20242', 
    'GSE22595', 
    'GSE37988', 
    'GSE17448', 
    'GSE36642', 
    'GSE26126', 
    'GSE34035', 
    'GSE28746',
    'GSE20236',
    'GSE19711',
    'GSE37008',
    'GSE36812',
    'GSE34257',
    'GSE38608',
    'GSE38291',
    'GSE36166',
    'GSE63384',
    'GSE59274',
    'GSE57285',
    'GSE56606',
    'GSE49908',
    'GSE49907',
    'GSE49905',
    'GSE49904'
]

geo_data_450k = [
    'GSE90124', 
    'GSE115797',
    'GSE99624', 
    'GSE108213',
    'GSE92767', 
    'GSE69176',
    'GSE40360',
    'GSE59157', 
    'GSE42861',
    'GSE77241',
    'GSE148000'
]      

array_express_27k = [
    'E-GEOD-43256',
    'E-GEOD-64940',
    'E-MTAB-2344',
    'E-GEOD-62867',
    'E-GEOD-44763',
    'E-GEOD-48988',
    'E-GEOD-58119',
    'E-GEOD-58045',
    'E-GEOD-57484',
    'E-GEOD-54211',
    'E-GEOD-56342',
    'E-GEOD-27044',
    'E-GEOD-36194',
    'E-GEOD-21232',
    'E-GEOD-32867',
    'E-GEOD-30759',
    'E-GEOD-30758',
    'E-GEOD-32396',
    'E-GEOD-31979',
    'E-MTAB-625',
    'E-MTAB-487'
]

array_express_450k = [
    'E-MTAB-2372',
    'E-GEOD-73832',
    'E-GEOD-71678',
    'E-GEOD-71245',
    'E-GEOD-83334',
    'E-GEOD-75248',
    'E-GEOD-77955',
    'E-GEOD-67705',
    'E-GEOD-77445',
    'E-GEOD-79056',
    'E-GEOD-72556',
    'E-GEOD-52068',
    'E-GEOD-74738',
    'E-GEOD-76105',
    'E-GEOD-65638',
    'E-GEOD-71955',
    'E-GEOD-63106',
    'E-GEOD-73377',
    'E-GEOD-56515',
    'E-GEOD-73103',
    'E-GEOD-67024',
    'E-GEOD-72338',
    'E-GEOD-59457',
    'E-GEOD-64511',
    'E-GEOD-64495',
    'E-GEOD-59509',
    'E-GEOD-67444',
    'E-GEOD-62219',
    'E-GEOD-51954',
    'E-GEOD-52588',
    'E-GEOD-36054',
    'E-GEOD-50660',
    'E-GEOD-61259',
    'E-GEOD-61258',
    'E-GEOD-61257',
    'E-GEOD-61454',
    'E-GEOD-61380',
    'E-GEOD-61107',
    'E-GEOD-54690',
    'E-GEOD-49149',
    'E-GEOD-55438',
    'E-GEOD-53740',
    'E-GEOD-57767',
    'E-GEOD-49064',
    'E-GEOD-50759',
    'E-GEOD-56553',
    'E-GEOD-54399',
    'E-GEOD-53162',
    'E-GEOD-53128', 
    'E-GEOD-50498', 
    'E-GEOD-47513',
    'E-GEOD-49393',
    'E-GEOD-39004',
    'E-GEOD-51388',
    'E-GEOD-51032', 
    'E-GEOD-48325',
    'E-GEOD-44712',
    'E-GEOD-45461',
    'E-GEOD-40279',
    'E-GEOD-41169',
    'E-GEOD-32149',
    'E-GEOD-41826',
    'E-GEOD-42700',
    'E-GEOD-32146',
    'E-GEOD-30870',
    'E-GEOD-34639',
    'E-GEOD-63347', 
    'E-GEOD-59592'
]

tcga_all = [
    'TGCA_LUSC',
    'TGCA_THCA',
    'TGCA_HNSC',
    'TGCA_KIRC',
    'TGCA_KIRP',
    'TGCA_LUAD',
    'TGCA_PRAD',
    'TGCA_STAD',
    'TGCA_COAD',
    'TGCA_LIHC',
    'TGCA_UCEC',
    'TGCA_BRCA'
]

cancer_data = [
    'GSE32393',
    'GSE37988',
    'GSE26126',
    'GSE63384',
    'GSE59157',
    'E-GEOD-32867',
    'E-GEOD-30759',
    'E-GEOD-31979',
    'E-GEOD-77955',
    'E-GEOD-52068',
    'E-GEOD-49149',
    'E-GEOD-39004'
]

cancer_comparison = [
    'GSE53051',
]

reliability = [
    'GSE55763',
]

rejuvenation = [
    'GSE142439',
    'GSE116754',
    'GSE65214', 
    'GSE44430',
    'GSE45727',
    'GSE30653',
    'GSE37066',
    'GSE30456',
]

senescence_analysis = [
    'GSE91069',
    'GSE100249'
]

In [ ]:
data_27k = np.concatenate([np.array(geo_data_27k), np.array(array_express_27k)])
data_450k = np.concatenate([np.array(geo_data_450k), np.array(array_express_450k)])
tcga_all = np.array(tcga_all)
all_data = np.concatenate([data_27k, data_450k, tcga_all])

In [ ]:
#load gold standard values from Horvath's
gs = pd.read_csv('/content/drive/MyDrive/Altum_Age_Dataset/gold.csv')
gs_cpgs = np.array(gs.Name)
gs_values = np.array(gs.overallMeanByCpGacross50data)

In [ ]:
sizes = {}
for d in all_data:
  try:
    size = os.path.getsize('/content/drive/MyDrive/AltumAge Dataset/Organized Data/' + d + '.pkl')
    sizes[d]=size
  except Exception as _:
    continue
sorted_datasets = sorted(list(sizes.keys()), key=lambda k: sizes[k], reverse=False)

In [ ]:
print(sorted_datasets)
print([sizes[k] for k in sorted_datasets])

['E-GEOD-43256', 'E-GEOD-21232', 'GSE22595', 'E-GEOD-30759', 'E-GEOD-32396', 'E-GEOD-31979', 'GSE17448', 'E-GEOD-62867', 'E-GEOD-54211', 'GSE38291', 'E-GEOD-57484', 'TGCA_STAD', 'GSE32393', 'TGCA_UCEC', 'E-MTAB-2344', 'TGCA_LIHC', 'GSE63384', 'GSE38608', 'E-GEOD-56342', 'GSE57285', 'TGCA_HNSC', 'TGCA_KIRP', 'TGCA_PRAD', 'GSE36166', 'E-GEOD-44763', 'TGCA_LUAD', 'GSE36812', 'TGCA_THCA', 'GSE59274', 'GSE20242', 'GSE49908', 'TGCA_LUSC', 'E-MTAB-625', 'GSE37988', 'E-GEOD-32867', 'E-MTAB-487', 'TGCA_COAD', 'GSE26126', 'GSE49904', 'GSE49905', 'GSE49907', 'GSE34257', 'GSE28746', 'TGCA_BRCA', 'GSE20236', 'GSE56606', 'GSE37008', 'GSE25892', 'GSE36642', 'E-GEOD-30758', 'GSE38873', 'GSE27317', 'E-GEOD-58045', 'E-GEOD-48988', 'GSE34035', 'E-GEOD-64940', 'E-GEOD-39004', 'E-GEOD-47513', 'E-GEOD-58119', 'E-GEOD-77955', 'TGCA_KIRC', 'E-GEOD-65638', 'E-GEOD-48325', 'E-GEOD-54690', 'E-GEOD-49064', 'E-GEOD-53162', 'E-GEOD-45461', 'E-GEOD-74738', 'E-GEOD-56553', 'E-GEOD-27044', 'GSE15745', 'E-GEOD-49149', 

In [ ]:
print(sizes["E-GEOD-77955"]/1024/1024)
i=sorted_datasets.index("E-GEOD-77955")
print(f"{i} / {len(sorted_datasets)}")

63.920108795166016
59 / 140


In [ ]:
!pip3 install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 5.0 MB/s 


In [ ]:
#normalize BMIQ
failed_datasets = []
count = 0
data = all_data
dfs = []
for dataset in sorted_datasets[0:2]:
  print(dataset)
  with open('/content/drive/MyDrive/Altum_Age_Dataset/data_norm_pkl/' + dataset + '.pkl', "rb") as fh:
    df = pickle.load(fh)
    dfs.append(df)

E-GEOD-43256
E-GEOD-21232


In [ ]:
age = None
gender= None
df = None
i = 0
for dataset in sorted_datasets[56:58]:
  i+=1
  print(i)
  if df is not None:
    print(f"Shape of the accumulated df: {df.shape}")
  with open('/content/drive/MyDrive/Altum_Age_Dataset/data_norm_pkl/' + dataset + '.pkl', "rb") as fh:
    new_df = pickle.load(fh)
  print(f"Shape of the new df: {new_df.shape}")
  if age is None:
    age = new_df["age"]
    print(age)
    gender = new_df["gender"]
    df = new_df
    continue

  age = age.append(new_df["age"])
  gender = gender.append(new_df["gender"])
  df = df.append(new_df)

# df.to_pickle('/content/drive/MyDrive/Altum_Age_Dataset/concatenated_dataframes.pkl')


  

1
Shape of the new df: (10, 485581)
0    51
1    93
2    43
3    73
4    45
5    37
6    55
7    44
8    30
9    39
Name: age, dtype: int64
2
Shape of the accumulated df: (10, 485581)
Shape of the new df: (12, 485581)


In [ ]:
df.to_pickle('/content/drive/MyDrive/Altum_Age_Dataset/56concatenated_dataframes.pkl')

In [ ]:
small_df = pd.read_pickle('/content/drive/MyDrive/Altum_Age_Dataset/56concatenated_dataframes.pkl')

In [ ]:
c1 = small_df.columns
c2 = df.columns
c3 = [column for column in c2 if column in c1]

In [ ]:
filtered_df = df[c3]

In [ ]:
filtered_df.to_pickle('/content/drive/MyDrive/Altum_Age_Dataset/filtered.pkl')

In [ ]:
small_df = small_df.append(filtered_df)
small_df.shape

(3720, 27582)

In [ ]:
filled = small_df.fillna(small_df.mean())
filled.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


(3720, 27582)

In [ ]:
print(filled.iloc[1:10,1:10])

        dataset       age gender  cg00000292  cg00002426  cg00003994  \
1  E-GEOD-43256 -0.536986      M    0.635999    0.249271    0.112311   
2  E-GEOD-43256 -0.613699      F    0.629611    0.276873    0.121368   
3  E-GEOD-43256 -0.536986      M    0.599255    0.186925    0.087592   
4  E-GEOD-43256 -0.652055      M    0.646849    0.212785    0.094603   
5  E-GEOD-43256 -0.594521      M    0.635511    0.255285    0.131895   
6  E-GEOD-43256 -0.556164      M    0.541424    0.180490    0.089318   
7  E-GEOD-43256 -0.652055      F    0.607095    0.131159    0.115193   
8  E-GEOD-43256 -0.594521      F    0.547574    0.176905    0.088136   
9  E-GEOD-43256 -0.556164      F    0.561583    0.314600    0.065613   

   cg00005847  cg00006414  cg00007981  
1    0.238007    0.094221    0.047817  
2    0.183297    0.091414    0.048323  
3    0.289278    0.078023    0.052625  
4    0.161115    0.081073    0.056713  
5    0.202257    0.093039    0.055076  
6    0.316909    0.094164    0.040721  

In [ ]:

filled.to_pickle('/content/drive/MyDrive/Altum_Age_Dataset/filled58concatenated_dataframes.pkl')

In [ ]:
filled = pd.read_pickle('/content/drive/MyDrive/Altum_Age_Dataset/filled58concatenated_dataframes.pkl')

In [ ]:
filled.shape

(3720, 27582)

In [ ]:
ages = filled["age"]

In [ ]:
ages[ages > 0].count()

3030

In [ ]:
ages[ages<=0].count()

690

In [ ]:
filled.iloc[1:10,1:10]

,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,cg00000292,cg00000321,cg00000363
1,0.181474,0.934105,0.761884,0.231319,0.837773,0.686632,0.577074,0.196243,0.143737
2,0.195400,0.909971,0.814580,0.363555,0.870637,0.608305,0.564458,0.213448,0.263882
3,0.120937,0.913760,0.728934,0.149397,0.812411,0.689175,0.609310,0.217254,0.121976
4,0.188715,0.928642,0.778999,0.202129,0.861120,0.700264,0.676087,0.207252,0.110413
5,0.176530,0.924695,0.787795,0.204350,0.852668,0.676977,0.613335,0.218695,0.152930
6,0.212309,0.909297,0.772013,0.267726,0.844834,0.710755,0.622235,0.250582,0.166488
7,0.197081,0.922222,0.783349,0.246678,0.841939,0.738347,0.619107,0.231235,0.168945
8,0.204960,0.923100,0.738479,0.221379,0.839748,0.649268,0.658288,0.268140,0.189683
9,0.270279,0.927713,0.776280,0.268691,0.841070,0.645016,0.619114,0.230220,0.257910


In [ ]:
filled = df.fillna(df.mean())
filled.to_pickle('/content/drive/MyDrive/Altum_Age_Dataset/filled56concatenated_dataframes.pkl')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
